这个模拟是检测除了变化的OH、wetland和biomass对结果是否有影响（整体或者特定时间段）  
这个模拟结果存储在results\none_but_one中，实验设计是把某个驱动保留正常，其他驱动变成全时间段的平均值，看结果有何变化。

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pylab as plt
import seaborn as sns

In [3]:
from core_fct.mod_process import OSCAR
from core_fct.fct_loadP import load_all_param
from core_fct.fct_genMC import generate_config
from run_scripts.get_SSP_drivers import For_hist
from run_scripts.get_SSP_drivers import For_scen

loading primary parameters


In [4]:
var_keep = ['D_Eluc', 'D_Focean', 'D_Fland', 'D_Epf'] + ['tau_CH4', 'tau_N2O'] + ['RF_CH4','RF_O3t']+['D_O3t']+['f_kOH_O3P']+['f_kOH']+['D_Ewet','D_Epf_CH4','D_Ebb']

In [5]:
with xr.open_dataset('./init/Ini.nc') as TMP: 
    Ini = TMP.load()
with xr.open_dataset('./init/Par.nc') as TMP: 
    Par = TMP.load()
with xr.open_dataset('./init/For_contact.nc') as TMP: 
    For = TMP.load()
with xr.open_dataset('./init/Out_ref.nc') as TMP: 
    Out_ref = TMP.load()

In [6]:
ch4 = pd.read_excel('ex_data/concentration/ch4.xlsx')

emi = pd.read_excel('ex_data/emi/emi.xlsx')[:-2]
oh = pd.read_excel('ex_data/oh/oh.xlsx')
wetland = pd.read_excel('ex_data/wetland/wetland.xlsx')
biomass = pd.read_excel('ex_data/biomass/biomass.xlsx')

In [7]:
emi_list = ['ceds','edgar','gains']
oh_list = ['ceds','edgar','Turner','Skeie','Nicely','He']
wetland_list = ['Skeie','Chandra','Zhang','Wetcharts']
biomass_list =['biomass']

In [8]:
# Ini = Out_hist.sel(year=1981, drop=True)
ref = []
##### 替换驱动数据并模拟 ####
output = []
n = 0
for var_emi in emi_list:
    For_contact = For.copy(deep=True)
    For_contact = xr.merge([For_contact,Out_ref.f_kOH.mean('config').copy(deep=True),Out_ref.D_Ewet.mean('config').copy(deep=True),Out_ref.D_Ebb.mean('config').copy(deep=True)])
    ### emission ###
    emi_temp = emi[var_emi].values*(1/16*12/1000)
    for i in range(len(emi_temp)):
        For_contact.E_CH4[i,:] *= 0
        For_contact.E_CH4[i,0] += np.mean(emi_temp)    
    
    ### biomass ###     
    for var_biomass in biomass_list:
        biomass_temp = biomass[var_biomass].values
        for year in np.arange(1981,2018+1):
            For_contact.D_Ebb.sel(spc_bb='CH4').sel(year=year).values *= 0
            For_contact.D_Ebb.sel(spc_bb='CH4').sel(year=year).isel(reg_land=0).isel(bio_land=0).values += np.mean(biomass_temp)     
    

        #### oh ###
        for var_oh in oh_list:
            # print('-'+var_oh)
            oh_temp = oh[var_oh].values
            For_contact.f_kOH.values = oh_temp      
            ### wetland ###     
            for var_wetland in wetland_list:
                # print('--'+var_wetland)
                wetland_temp = wetland[var_wetland].values
                for year in np.arange(1981,2018+1):
                    For_contact.D_Ewet.sel(year=year).values *= 0
                    For_contact.D_Ewet.sel(year=year).isel(reg_land=0).values += np.mean(wetland_temp)
                # ### run ###
                n += 1
                text = var_emi+' '+var_oh+' '+var_wetland+' '+var_biomass
                print(text)
                # print(n)
                Out_run = OSCAR(Ini=Ini, Par=Par, For=For_contact, var_keep=var_keep, nt=4)
                Out_run.to_netcdf('results/none_but_one/oh/' + str(n)+'_ref.nc',encoding={var:{'zlib':True, 'dtype':np.float32} for var in Out_ref})
                ref.append(Out_run.D_CH4.mean('config'))
ref = np.array(ref)

ceds ceds Skeie biomass
OSCAR_v3 running
year = 2018 (nt = 4)
total running time: 1.6 minutes
ceds ceds Chandra biomass
OSCAR_v3 running
year = 2018 (nt = 4)
total running time: 1.6 minutes
ceds ceds Zhang biomass
OSCAR_v3 running
year = 2018 (nt = 4)
total running time: 1.6 minutes
ceds ceds Wetcharts biomass
OSCAR_v3 running
year = 2018 (nt = 4)
total running time: 1.6 minutes
ceds edgar Skeie biomass
OSCAR_v3 running
year = 2018 (nt = 4)
total running time: 1.6 minutes
ceds edgar Chandra biomass
OSCAR_v3 running
year = 2018 (nt = 4)
total running time: 1.6 minutes
ceds edgar Zhang biomass
OSCAR_v3 running
year = 2018 (nt = 4)
total running time: 1.6 minutes
ceds edgar Wetcharts biomass
OSCAR_v3 running
year = 2018 (nt = 4)
total running time: 1.6 minutes
ceds Turner Skeie biomass
OSCAR_v3 running
year = 2018 (nt = 4)
total running time: 1.6 minutes
ceds Turner Chandra biomass
OSCAR_v3 running
year = 2018 (nt = 4)
total running time: 1.6 minutes
ceds Turner Zhang biomass
OSCAR_v3 run